<a href="https://colab.research.google.com/github/mumanoha/genai/blob/main/Copy_of_GAB_Check_Documents_in_Data_Store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TLDR
Simple notebook that uses the Cloud DiscoveryEngine API to check a Data Store for indexed docs.

* Author: [pmarlow@](https://moma.corp.google.com/person/pmarlow)
* Date: 07/17/23

In [ ]:
!pip install google-cloud-discoveryengine --quiet

from google.colab import auth
from google.auth import default

auth.authenticate_user()
creds, _ = default()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.8/431.8 kB 2.8 MB/s eta 0:00:00


MessageError: ignored

# DiscoveryEngine Client and Helper Functions

In [ ]:
from google.cloud import discoveryengine_v1
from typing import List

def list_documents(project_id: str, location: str, datastore_id: str):
  """Gets a list of docs in a datastore."""
  client = discoveryengine_v1.DocumentServiceClient()

  request = discoveryengine_v1.ListDocumentsRequest(
      parent=f'projects/{project_id}/locations/{location}/collections/default_collection/dataStores/{datastore_id}/branches/0'
  )

  res = client.list_documents(request=request)

  return list(res)

def list_indexed_urls(project_id: str, location: str, datastore_id: str):
  """Get the list of docs in datastore, then parse to only urls."""
  docs = list_documents(project_id, location, datastore_id)
  urls = [doc.content.uri for doc in docs]

  return urls

def search_url(urls: List[str], url: str):
  """Searches a url in a list of urls."""
  for item in urls:
    if url in item:
      print(item)

# USER INPUTS
You can find your `datastore_id` by going following these steps:
1. Click on Gen App Builder
2. Select your App / Engine - https://screenshot.googleplex.com/9J4nfmfDhpvU7PK
3. Select your Available Data Store - https://screenshot.googleplex.com/4j3XxCFPoq3zXLo
4. Find your Data Store ID - https://screenshot.googleplex.com/Ad5cGZQbVEbdqfm

In [ ]:
project_id = 'pmarlow-ccai-dev'
location = 'global'
datastore_id = 'yeti-chatbot_1688158303521'

# List Documents (in Data Store)
If you run this and it errors out with `IndexError: list index out of range` that means that there are no docs in your Data Store yet.  
So either indexing has not completed, or it has failed.  

TODO: Add method for finding failure reasons

In [ ]:
docs = list_documents(project_id, location, datastore_id)
docs[0]

name: "projects/772105163160/locations/global/collections/default_collection/dataStores/yeti-chatbot_1688158303521/branches/0/documents/002f4c29ea9dfa7a83af8c23dd0b1816"
id: "002f4c29ea9dfa7a83af8c23dd0b1816"
schema_id: "default_schema"
struct_data {
}
parent_document_id: "002f4c29ea9dfa7a83af8c23dd0b1816"
content {
  mime_type: "text/html"
  uri: "https://www.yeti.com/shop-by-color/navy/21070140015.html?bvstate=pg:2/ct:r"
}

# List Indexed URLs

In [ ]:
urls = list_indexed_urls(project_id, location, datastore_id)
urls[0]

'https://www.yeti.com/shop-by-color/navy/21070140015.html?bvstate=pg:2/ct:r'

# Search Indexed URLs

In [ ]:
search_url(urls, 'https://www.yeti.com/bags/luggage')

https://www.yeti.com/bags/luggage/26010000103.html?bvstate=pg:2/ct:r
https://www.yeti.com/bags/luggage/crossroads-luggage-29in.html
https://www.yeti.com/bags/luggage/18060131085.html?bvstate=pg:2/ct:r
https://www.yeti.com/bags/luggage/crossroads-luggage-29in.html?bvstate=pg:2/ct:r
https://www.yeti.com/bags/luggage/crossroads-luggage-22in.html
https://www.yeti.com/bags/luggage


In [ ]:
search_url(urls, 'tundra-250')

https://www.yeti.com/coolers/hard-coolers/tundra/tundra-250.html
https://www.yeti.com/coolers/hard-coolers/tundra/tundra-250.html?bvstate=pg:2/ct:r
